In [6]:
import pickle

with open('promo_pickle.pkl','rb') as pkl:
    df = pickle.load(pkl)

In [7]:
# dtypes are preserved
df.dtypes

file_id                               object
chainid                                int64
subchainid                             int64
storeid                                int64
bikoretno                              int64
priceupdatedate               datetime64[ns]
itemcode                               int64
isgiftitem                             int64
rewardtype                             int64
allowmultiplediscounts                 int64
promotionid                            int64
promotiondetails                      object
additionalrestrictions                object
clubs                                 object
discountrate                         float64
discounttype                           int64
minqty                               float64
promotiondescription                  object
weightunit                             int64
promotionupdatedate           datetime64[ns]
minnoofitemofered                      int64
isweightedpromo                        int64
promotioni

In [10]:
all_columns = list(df.columns)

ignore = [
    'bikoretno' # מס ביקורות שאני לא יודע להסביר
]

index = [
    'promotionupdatedate','priceupdatedate' #  שעת עדכון המבצע בנוגע למוצר (לבדוק האם משלימים)
    'itemcode', # מספר הברקוד של המצרך
    'itemtype' #  האם ברקוד פנימי
]
promo_start_end = [
    'promotionstartdatetime', # תיאור המבצע
    'promotionenddatetime'# סוף המבצע
]

store_columns = [
    'file_id', # קובץ מקור
    'chainid', # קוד הרשת
    'subchainid', # קוד תת-רשת
    'storeid', # מספר החנות
]


deal_condition = [
    'minqty',# כמות מזערית להשתתפות במבצע  
    'maxqty',# הגבלת כמות לרכישה במסגרת המבצע
    'minnoofitemsoffered','minnoofitemofered',  # (לבדוק האם משלימים)המספר המזערי של המצרכים המוצעים במבצע בחנות
    'minpurchaseamnt','minpurchaseamount', # סכום הרכישה המזערי(לבדוק האם משלימים) 
    'additionalrestrictions' # הגבלות נוספות על המבצע
]



flag_columns = [
    'isgiftitem',
    'isweightedpromo',
    'allowmultiplediscounts' # כפל מבצעים
]
promo_columns = [
    'promotionid', # מזהה מבצע
    'discountedprice', # המחיר הכולל של המבצע
    'discountrate', # שיעור ההנחה
    'discountedpricepermida',
    'discounttype','rewardtype' # 0 precentage discount, 1 group deal
    'promotiondescription','promotiondetails',  # תיאור
    'giftsitems',
]


quntity_columns = [
    'weightunit',
]


customer_with_deal = [
    'clubid','clubs', # האוכלוסייה שאליה מכוון המבצע
    'promotionitems'
]

rest = list(set(all_columns) - set(promo_start_end+index+promo_columns+deal_condition+ignore+store_columns+quntity_columns+customer_with_deal+promo_columns+flag_columns))
rest

['remark',
 'additionalsgiftcount',
 'rewardtype',
 'additionalsminbasketamount',
 'additionalstotals',
 'additionalscoupon',
 'remarks',
 'priceupdatedate']

In [79]:
# bikoretno
df = df.drop(columns=['bikoretno'])

In [ ]:
# discounttype

In [41]:
# check if this type is in use in all chains
discounttype_on_chain_id = df[df.discounttype == 1]['chainid'].unique()
discounttype_on_chain_id

array([7290058134977, 7290027600007, 7290058173198, 7290058197699,
       7290058108879, 7290875100001, 7290058198450, 7290058160839,
       7290058159628, 7290696200003])

In [42]:
discounttype_off_chain_id = df[df.discounttype == 0]['chainid'].unique()
discounttype_off_chain_id

array([7290058134977, 7290027600007, 7290058173198, 7290058197699,
       7290058108879, 7290875100001, 7290058198450, 7290058160839,
       7290058159628, 7290696200003])

In [44]:
set(discounttype_off_chain_id) - set(discounttype_on_chain_id) == set()
# seem like those chains are using the type

True

In [45]:
discounttype_nan_chain_id = df[df.discounttype == -1]['chainid'].unique()
discounttype_nan_chain_id

array([7290172900007, 7290058140886, 7291059100008, 7290873255550,
       7290027600007, 7290725900003, 7290876100000, 7290103152017,
       7291056200008, 7290639000004, 7290803800003, 7290526500006,
       7290055700014, 7290644700005, 7290492000005, 7290700100008,
       7290785400000])

In [71]:
list(set(discounttype_nan_chain_id) & set(discounttype_off_chain_id))
# this chain use also the -1 as 'unknown'?

[7290027600007]

In [77]:
df.loc[(df.discounttype == -1) & (~df.chainid.isin(list(set(discounttype_nan_chain_id) & set(discounttype_off_chain_id)))
),'discounttype'] = -2 